## Tao training dataset tu du lieu ve tinh

In [44]:
import geopandas as gpd
import rasterio

def shp2txt(fp_tif, fp_shp, out_fp_txt):
    # Đọc dữ liệu từ shapefile
    gdf = gpd.read_file(fp_shp)
    # Đọc thông tin ảnh (góc trên bên trái, kích thước pixel)
    with rasterio.open(fp_tif) as dataset:
        img_transform = dataset.transform  # Góc trên bên trái và kích thước pixel của ảnh

    # Chuyển đổi tọa độ từ đơn vị địa lý sang pixel trên ảnh
    def convert_to_pixel_coords(x, y):
        row, col = ~img_transform * (x, y)  # Ánh xạ tọa độ địa lý sang pixel
        return int(col), int(row)  # Trả về tọa độ pixel (cột, hàng)

    # Ghi thông tin vào file txt với định dạng YOLO
    # out_fp_txt = r'/home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/code/data/split_data/output.txt'

    with open(out_fp_txt, 'w') as f:
        for index, row in gdf.iterrows():
            x_min, y_min, x_max, y_max = row['geometry'].bounds
            class_id = 0  # Mã lớp tương ứng với đối tượng

            # Chuyển đổi tọa độ từ đơn vị địa lý sang pixel trên ảnh
            x_min, y_min = convert_to_pixel_coords(x_min, y_min)
            x_max, y_max = convert_to_pixel_coords(x_max, y_max)
            
            # chuyen ve dung dinh dang toa do pixel
            x_min, y_min, x_max, y_max = y_min, x_max, y_max, x_min
            # print(x_min, y_min, x_max, y_max)
            # Tính toán tọa độ và kích thước hộp giới hạn theo định dạng YOLO
            image_width, image_height = dataset.width, dataset.height
            x_center_yolo = (x_min + x_max) / (2 * image_width)
            y_center_yolo = (y_min + y_max) / (2 * image_height)
            width_yolo = (x_max - x_min) / image_width
            height_yolo = (y_max - y_min) / image_height

            # Ghi vào file txt
            line = f"{class_id} {x_center_yolo:.6f} {y_center_yolo:.6f} {width_yolo:.6f} {height_yolo:.6f}\n"
            # print(line)
            f.write(line)
    print(f"File {out_fp_txt} đã được tạo.")

# Chuyen tif sang jpg

In [47]:
import os
from PIL import Image
def tif2jpg(fp_tif, fp_jpg):
    # Đọc ảnh từ định dạng TIFF
    image = Image.open(fp_tif)

    # Chuyển đổi và lưu ảnh dưới định dạng JPEG
    image.save(fp_jpg)
    print(f'Done {os.path.basename(fp_jpg)}')


In [43]:
# test tif2jpg
# fp_tif = r'/home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/data_faster-rnn/image_crop_box/tmp/gen_TauBien_Original_3band_cut_256_stride_128_V2/images_data/train/images/DaNang_23_07_2009_LowAccuracy_0_2.tif'
# fp_jpg  =r'/home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/data_faster-rnn/image_crop_box/tmp/gen_TauBien_Original_3band_cut_256_stride_128_V2/images_data/train/a.jpg'
# tif2jpg(fp_tif, fp_jpg)

In [49]:
import os, glob
data_set = r'/home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/data_faster-rnn/image_crop_box/tmp/gen_TauBien_Original_3band_cut_256_stride_128_V2/images_data'
list_set = ['train', 'val']
image_size = 256
for item in list_set:
    dir_tif = f'/home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/data_faster-rnn/image_crop_box/tmp/gen_TauBien_Original_3band_cut_256_stride_128_V2/images_data/{item}/images'
    dir_shp =  f'/home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/data_faster-rnn/image_crop_box/tmp/gen_TauBien_Original_3band_cut_256_stride_128_V2/images_data/{item}/images_shape'
    out_dir_txt = f'/home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/code/YOLO/Custom/TrainYOLOv7/train_data/{item}_ship_{image_size}/labels'
    os.makedirs(out_dir_txt, exist_ok=True)
    out_dir_jpg = f'/home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/code/YOLO/Custom/TrainYOLOv7/train_data/{item}_ship_{image_size}/images'
    os.makedirs(out_dir_jpg, exist_ok=True)
    
    list_tif = glob.glob(os.path.join(dir_tif, '*.tif'))
    print(len(list_tif))
    for fp_tif in list_tif:
        fp_shp = os.path.join(dir_shp, os.path.basename(fp_tif).replace('.tif','.shp'))
        out_fp_txt = os.path.join(out_dir_txt, os.path.basename(fp_tif).replace('.tif','.txt'))
        shp2txt(fp_tif, fp_shp, out_fp_txt)
        
        fp_jpg = os.path.join(out_dir_jpg, os.path.basename(fp_tif).replace('.tif','.jpg'))
        tif2jpg(fp_tif, fp_jpg)
    

497
File /home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/code/YOLO/Custom/TrainYOLOv7/train_data/train_ship_256/labels/DaNang_23_07_2009_LowAccuracy_4_80.txt đã được tạo.
Done DaNang_23_07_2009_LowAccuracy_4_80.jpg
File /home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/code/YOLO/Custom/TrainYOLOv7/train_data/train_ship_256/labels/DaNang_23_07_2009_LowAccuracy_0_2.txt đã được tạo.
Done DaNang_23_07_2009_LowAccuracy_0_2.jpg
File /home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/code/YOLO/Custom/TrainYOLOv7/train_data/train_ship_256/labels/DaNang_23_07_2009_LowAccuracy_0_6.txt đã được tạo.
Done DaNang_23_07_2009_LowAccuracy_0_6.jpg
File /home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/code/YOLO/Custom/TrainYOLOv7/train_data/train_ship_256/labels/DaNang_23_07_2009_LowAccuracy_0_7.txt đã được tạo.
Done DaNang_23_07_2009_LowAccuracy_0_7.jpg
File /home/skm/SKM16/IMAGE/ZZ_ZZ/TauBien/code/YOLO/Custom/TrainYOLOv7/train_data/train_ship_256/labels/DaNang_23_07_2009_LowAccuracy_1_1033.txt đã được tạo.
Done DaNang_23_07_2009_LowAccuracy_1_1033.jpg
Fil

## xong thi nho cop anh PNG vao de lm mau